## 3조 프로젝트
### 주제: 데이터로 추적하는 넷플릭스 구독자들의 이탈 신호와 유지 전략

In [ ]:
### **화면 1: 넷플릭스 구독자 현황**
- **상단 Metric:** 전체 활성 사용자 수(DAU), 평균 구독 기간(LTV), 이번 달 이탈률(Churn Rate).
- **멤버십별 비교:** 베이식 vs 프리미엄 등 요금제별 이탈률 차이 시각화.
- **인사이트:** "광고형 요금제 가입자가 일반 요금제보다 이탈률이 높은가?"에 대한 답을 제공합니다.

In [ ]:
%%writefile module/project.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# ======================================================== 1.페이지 설정 =============================================================

st.set_page_config(page_title="넷플릭스 대시보드", layout="wide", initial_sidebar_state="expanded")

# ======================================================== 2. 데이터 =================================================================
@st.cache_data 
def load_data():
    data = {
        'Month': pd.date_range(start='2025-01-01', periods=12, freq='MS'),
        'Subscribers': [1250, 1270, 1321, 1345, 1360, 1393, 1410, 1425, 1440, 1465, 1490, 1516],
        'Retention': [12.1, 12.5, 11.8, 13.0, 14.2, 13.5, 12.8, 13.2, 14.5, 15.0, 14.8, 15.2],
    }
    df = pd.DataFrame(data)

    df['Prev_Subscribers'] = df['Subscribers'].shift(1)
    df['Growth_Rate'] = ((df['Subscribers'] - df['Prev_Subscribers']) / df['Prev_Subscribers']) * 100

    age_df = pd.DataFrame({
        '연령대': ['10대 이하', '20대', '30대', '40대', '50대', '60대 이상'],
        '비중': [8, 22, 25, 24, 15, 6]
    })

    gender_df = pd.DataFrame({
        '성별': ['여성', '남성'],
        '비중': [55, 45]
    })

    return {
        'main_df': df,
        'age_df': age_df,
        'gender_df': gender_df
    }

data_dict = load_data()
in_df = data_dict['main_df']
age_df = data_dict['age_df']
gender_df = data_dict['gender_df']

# ======================================================== 3.사이드바 구성=============================================================
with st.sidebar:
    st.header("🔍 분석 설정")
    month_labels = [d.strftime('%Y년 %m월') for d in in_df['Month']]
    selected_month = st.selectbox("분석 월 선택", options=month_labels, index=0)
    
    # 요금제 필터 제거됨
    analysis = st.button("🚀 데이터 분석 실행", use_container_width=True)
    st.divider()
    # 안내 문구에서 요금제 정보 제거
    st.info(f"💡 [현재 설정] 기간: **{selected_month}**")

# ======================================================== 4. 메인화면 구성=============================================================

header_col1, header_col2 = st.columns([1.5, 6])
with header_col1:
    st.image('https://upload.wikimedia.org/wikipedia/commons/0/08/Netflix_2015_logo.svg', width=250)
    
with header_col2:
    st.title('넷플릭스 구독자 현황 분석')
    st.text('💡 데이터로 추적하는 넷플릭스 구독자들의 이탈 신호와 유지 전략')
    
st.divider()

# ======================================================== 5. 유저 프로필 ====================================================

st.subheader("📊 넷플릭스 독점 유저 프로필 (2025년 8월 기준)")
col_left, col_right = st.columns(2)

with col_left:
    fig_age = px.bar(
        age_df, x='비중', y='연령대', 
        orientation='h', text_auto='%',
        title="📅 연령분포: 3040 세대가 주력",
        color_discrete_sequence=['#F6B6B6']
    )
    st.plotly_chart(fig_age, use_container_width=True)

with col_right:
    fig_gender = px.pie(
        gender_df, values='비중', names='성별',
        hole=0.5, title="👫 성별비중: 여성 유저 과반수",
        color_discrete_sequence=['#2E4766', '#8C9DB1']
    )
    fig_gender.update_traces(textinfo='percent+label')
    st.plotly_chart(fig_gender, use_container_width=True)

st.divider()

# ======================================================== 6. 분석 로직 ==================================================

if analysis:
    month_num = int(selected_month.split(' ')[1].replace('월', ''))
    target_df = in_df.iloc[month_num - 1:month_num] 
    
    if not target_df.empty:
        latest_data = target_df.iloc[0]
        latest_mau = latest_data['Subscribers'] / 100  
        growth_rate = latest_data['Growth_Rate']


        st.subheader(f"📊 {selected_month} 분석 결과")

        col3, col4 = st.columns(2)
        with col3:
            delta_text = f"{growth_rate:.2f}% (전월 대비)" if pd.notnull(growth_rate) else "신규 데이터"
            st.metric(
                label="📈 월 가입자 수", 
                value=f"{latest_mau:.2f} M", 
                delta=delta_text
            )
            st.caption("(가입자 수): 전체 체급 지표")
            
        with col4:
            st.metric(label="⏳ 유지 기간", value=f"{latest_data['Retention']}개월", delta="0.5개월")
            st.caption("(유지 기간): 수익성 지표")
        st.divider()